## Regression :
    All in all this notebook isn't really interesting as is but : Regression analysis of our current dataset with the number of urls for each videos, this workframe while be replicated when we will have formally defined what a sponsor is :

In [1]:
import re
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, LongType, StringType, DateType, ArrayType, BooleanType

from pyspark.sql import SparkSession
import pyspark as ps
import pyspark.sql.functions as F

import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

config = ps.SparkConf().setAll([
    ('spark.network.timeout', '3601s'),
    ('spark.executor.heartbeatInterval', '3600s'),
])
sc = ps.SparkContext('local', '', conf=config)
spark = SparkSession(sc)

22/11/18 12:43:56 WARN Utils: Your hostname, LAPTOP-8QFB5E0N resolves to a loopback address: 127.0.1.1; using 172.17.221.138 instead (on interface eth0)
22/11/18 12:43:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 12:43:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Data prep :
-----------------------------------------------------------------------------------------------------

In [2]:
PATH_METADATA_W_URLS = 'data/yt_metadata_en_urls.parquet'

In [3]:
df=spark.read.parquet(PATH_METADATA_W_URLS, sep=',')

-------------------------------------------------------------------------------------------------------------------------------

In [4]:
df.head()

Row(categories='Howto & Style', channel_id='UCROB2-0bJEcwiP059oNil_Q', crawl_date=datetime.date(2019, 11, 17), dislike_count=None, display_id='iICAtB8ViFM', duration=1365, like_count=None, tags='makeup geek,kathleen lights,makeup,wannamakeup,sephora,by terry,ulta,korres,chanel,haul', title='Makeup Haul | Chanel, MAC, Makeup Geek, Sephora & more', upload_date=datetime.date(2016, 8, 22), view_count=2813, urls=['http://go.magik.ly/ml/1fqi/', 'http://go.magik.ly/ml/1fqk/', 'http://go.magik.ly/ml/1fqo/', 'http://go.magik.ly/ml/1fql/', 'http://go.magik.ly/ml/1fqm/', 'http://go.magik.ly/ml/1fqr/', 'http://go.magik.ly/ml/1fqp/', 'http://www.ebates.com/rf.do?referreri...', 'http://go.magik.ly/ml/1fqn/', 'http://go.magik.ly/ml/1fqj/', 'http://go.magik.ly/ml/1fqs/', 'http://go.magik.ly/ml/167n/', 'http://go.magik.ly/ml/1fqq/', 'https://www.octoly.com/creators?cref=hato9'], urls_count=14, has_urls='true')

In [4]:
df.select(col('like_count')).show()

+----------+
|like_count|
+----------+
|      null|
|      null|
|      null|
|      null|
|      null|
|        87|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|        50|
|       454|
|      null|
|      null|
|      null|
|       214|
|        97|
+----------+
only showing top 20 rows



In [4]:
df = df.fillna(0,subset='dislike_count') # if we want to work with the dislike/like count we would need to change all "nuLL" values to 0
df = df.fillna(0,subset='like_count')

In [5]:
df = df.withColumn('like_per_view', df.like_count / df.view_count)

In [50]:
df.select(col('like_per_view')).show()

+--------------------+
|       like_per_view|
+--------------------+
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
| 0.10661764705882353|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
| 0.05714285714285714|
|0.039564270152505444|
|                 0.0|
|                 0.0|
|                 0.0|
| 0.08788501026694046|
| 0.10948081264108352|
+--------------------+
only showing top 20 rows



In [6]:
df = df.fillna(0,subset='like_per_view')

In [29]:
df.filter(col("like_per_view")==0).count() # see the number of videos where the ratio is 0

7094044

In [21]:
print((df.count(), len(df.columns))) # it represent less than 10 % of our dataset

(72924794, 15)


In [7]:
#we can drop the videos where like/view ratio is zero since there's a near 0 chance that we can consider them as sponsored
df = df.where(df.like_per_view>0)

In [8]:
df = df.withColumn('dislike_per_view', df.dislike_count / df.view_count)
df = df.fillna(0,subset='dislike_per_view')

In [55]:
df.head()

Row(categories='Howto & Style', channel_id='UCROB2-0bJEcwiP059oNil_Q', crawl_date=datetime.date(2019, 11, 17), dislike_count=1, display_id='4e3A7ohrWZ0', duration=426, like_count=87, tags='urban decay,foundation,review,makeup,sephora,ulta,wannamakeup,demo,first impressions,all nighter,too faced,colourpop', title='Urban Decay All Nighter Foundation | Demo and Review', upload_date=datetime.date(2016, 8, 11), view_count=816, urls=['http://www.ebates.com/rf.do?referreri...', 'https://www.octoly.com/creators?cref=hato9'], urls_count=2, has_urls='true', like_per_view=0.10661764705882353, dislike_per_view=0.0012254901960784314)

# Model fitting
-----------------------------------------------------------------------------------------------------
Attempt at using the spark library of regression, not yet opreationnal

-------------------------------------------------------------------------------------------------------------------------------
Let's use a statsmodel approach for large data sets (https://www.statsmodels.org/stable/large_data.html)

In [ ]:
# Run this cell to write the parquet file regression_urls with the columns you want to fit on :
df2 = df.select(col('urls_count'),col('like_per_view'))
df2.write.parquet('regression_urls.parquet')

In [9]:
import pyarrow as pa
import pyarrow.parquet as pq
import statsmodels.formula.api as smf

class DataSet(dict):
    def __init__(self, path):
        self.parquet = pq.ParquetDataset(path)

    def __getitem__(self, key):
        try:
            return self.parquet.read([key]).to_pandas()[key]
        except:
            raise KeyError

df_urls = DataSet('regression_urls.parquet')

In [10]:
mod = smf.ols('like_per_view ~ urls_count', data=df_urls)

In [11]:
import numpy as np

np.random.seed(2)
res = mod.fit()

In [12]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          like_per_view   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5037.
Date:                Fri, 18 Nov 2022   Prob (F-statistic):               0.00
Time:                        12:49:36   Log-Likelihood:             1.5755e+07
No. Observations:            65830750   AIC:                        -3.151e+07
Df Residuals:                65830748   BIC:                        -3.151e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0232   2.66e-05    872.818      0.0

We can atleast see that we have a positive coefficient for urls_count which would seems coherent with the fact that popular videos (thus with more likes) tend to have a more worked-through description (more descriptive etc) which means also more links it them.